In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
from netCDF4 import num2date, date2num, date2index
import datetime as dt
import pandas as pd
from StringIO import StringIO
import xarray as xr
from scipy import stats
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import cartopy.feature as cfeature
import seaborn as sns
from datetime import datetime

/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/bgh2-ra/anaconda2/lib/python2.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
nameCSV = 'melville_ACTUAL_FILEEE.csv'
df = pd.read_csv(str(nameCSV), sep=r'\s+', engine='c', header=0, na_filter=False, \
                               dtype=np.float, low_memory=False)

data = df.sort_values(by=['tt'])

lat = data.values[:,0]
lon = data.values[:,1]
wh = data.values[:,2]
tt = data.values[:,3]
qc = data.values[:,4]
back =data.values[:,5]
ws = data.values[:,6]
len(lat)

2998

## Calculate mean value of each satellite track 

In [3]:
days = [] #daystart
for k in range(len(tt)):
       
    t1 = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
    if k == 0: #get day 0. Then else: every day after that
        days.append(0)
        dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days.append(k)
            it += 1
            dd = netCDF4.num2date(tt[k],u'days since 1985-01-01 00:00:00 UTC')
            
            
print 'Number of Satellite Tracks:', len(tt[days])

Number of Satellite Tracks: 2639


## Make ndarray(alltime) and list(days2) that has dates

In [4]:
alltime = num2date(tt[:],u'days since 1985-01-01 00:00:00 UTC')

In [5]:
for k in range(len(days)):
    print wh[days[k]], alltime[k]

1.3633999824523926 1985-04-05 21:27:30.469790
1.3511250019073486 1985-04-14 09:26:31.513184
1.34530770778656 1985-05-07 11:06:14.088943
2.0326924324035645 1985-05-30 12:45:54.566106
1.6260000467300415 1985-06-02 12:54:42.804684
0.9227856993675232 1985-06-25 14:34:14.719868
1.345666766166687 1985-08-10 17:53:29.313367
2.313499927520752 1985-09-02 19:32:58.781247
2.4012999534606934 1985-09-05 19:41:45.054690
1.3407059907913208 1985-09-28 21:21:12.227023
0.32442858815193176 1985-10-16 11:10:26.379465
1.9190832376480105 1985-11-08 12:49:40.654948
1.4140626192092896 1985-11-14 00:39:52.969910
1.2756000757217407 1985-12-07 02:18:58.426560
0.5147500038146973 1986-01-19 17:56:10.249999
0.3118000030517578 1986-02-11 19:35:15.709374
0.5635000467300415 1986-03-12 09:03:50.362630
2.66937518119812 1986-04-04 10:42:45.532226
1.7990001440048218 1986-04-07 10:51:29.847656
0.3752500414848328 1986-04-27 12:21:35.900392
1.867833495140076 1986-04-30 12:30:16.296876
2.049000024795532 1986-05-20 14:00:18.82

1.7505335807800293 2005-02-26 00:08:22.788497
2.340199708938598 2005-03-06 22:46:55.713941
1.6154545545578003 2005-03-11 12:26:43.680664
4.180866718292236 2005-03-12 10:33:58.570772
2.64775013923645 2005-03-17 00:11:13.963500
2.6860671043396 2005-03-29 11:46:47.413021
1.37994122505188 2005-04-02 00:08:23.874704
2.1683332920074463 2005-04-10 01:12:33.017158
2.560272455215454 2005-04-15 12:26:49.145508
2.3433334827423096 2005-04-15 12:59:36.520833
1.6333333253860474 2005-04-21 00:11:21.150098
2.0456666946411133 2005-04-27 02:25:20.970214
2.436800241470337 2005-05-02 14:12:23.474331
2.2589335441589355 2005-05-07 00:08:30.575399
1.784400224685669 2005-05-14 03:38:11.057617
2.29223108291626 2005-05-19 15:25:11.090821
2.4900002479553223 2005-05-20 12:26:50.515137
2.635307788848877 2005-05-26 00:11:21.610197
1.6182667016983032 2005-05-31 04:50:56.377405
0.8724285364151001 2005-06-05 16:37:57.236328
1.4883685111999512 2005-06-11 00:08:28.771700
0.5945000052452087 2005-06-17 06:03:44.094922
0.5

2.0378890567355685 2014-10-08 01:48:52.543324
2.5016667683919267 2014-10-13 11:53:29.412326
3.4170769911545973 2014-10-17 10:17:42.111327
2.376571655273437 2014-10-17 23:47:23.236980
1.3007059097290041 2014-10-19 10:15:08.741268
1.4014167785644531 2014-10-21 10:12:42.251302
1.5229412527645336 2014-10-23 09:52:01.532986
1.2428571156093051 2014-10-26 21:48:27.681986
0.9726666344536676 2014-10-27 21:45:55.611979
2.142750024795532 2014-10-28 21:46:00.539063
2.4620834986368814 2014-11-02 07:50:33.765623
2.2813892364501958 2014-11-06 19:44:27.723957
2.3363159581234583 2014-11-12 05:49:05.845052
1.5612942190731274 2014-11-15 08:52:29.350644
1.4766000747680663 2014-11-16 17:42:59.255209
1.7006471157073977 2014-11-17 08:50:03.224393
1.4096666971842449 2014-11-19 08:47:34.027344
1.0540770017183745 2014-11-22 03:47:37.298177
0.6091111501057943 2014-11-22 20:25:54.264757
0.686400032043457 2014-11-24 20:23:23.617188
1.945333480834961 2014-11-24 20:23:32.107423
3.0062224070231114 2014-11-26 15:41:31

In [6]:
days2 = [] #daystart
for k in range(len(alltime)):
       
    t1 = alltime[k]
    if k == 0: #get day 0. Then else: every day after that
        days2.append(alltime[k])
        dd =alltime[k]
        it = 0
    else:
        if t1.day != dd.day: #if day 1 is not equal to day 2, then append
            #print dd.day,t1.day
            days2.append(alltime[k])
            it += 1
            dd = alltime[k]
            
            
print 'Number of Satellite Tracks:', len(tt[days])

Number of Satellite Tracks: 2639


In [7]:
days2

[datetime.datetime(1985, 4, 5, 21, 27, 30, 469790),
 datetime.datetime(1985, 4, 14, 9, 26, 31, 513184),
 datetime.datetime(1985, 5, 7, 11, 6, 14, 88943),
 datetime.datetime(1985, 5, 30, 12, 45, 54, 566106),
 datetime.datetime(1985, 6, 2, 12, 54, 42, 804684),
 datetime.datetime(1985, 6, 25, 14, 34, 14, 719868),
 datetime.datetime(1985, 8, 10, 17, 53, 29, 313367),
 datetime.datetime(1985, 9, 2, 19, 32, 58, 781247),
 datetime.datetime(1985, 9, 5, 19, 41, 45, 54690),
 datetime.datetime(1985, 9, 28, 21, 21, 12, 227023),
 datetime.datetime(1985, 10, 16, 11, 10, 26, 379465),
 datetime.datetime(1985, 11, 8, 12, 49, 40, 654948),
 datetime.datetime(1985, 11, 14, 0, 39, 52, 969910),
 datetime.datetime(1985, 12, 7, 2, 18, 58, 426560),
 datetime.datetime(1986, 1, 19, 17, 56, 10, 249999),
 datetime.datetime(1986, 2, 11, 19, 35, 15, 709374),
 datetime.datetime(1986, 3, 12, 9, 3, 50, 362630),
 datetime.datetime(1986, 4, 4, 10, 42, 45, 532226),
 datetime.datetime(1986, 4, 7, 10, 51, 29, 847656),
 datet

## WAVE PERIOD EQN

$ \epsilon = 3.25 \left( H_s^2 g^2/U^4 \right)^{0.31}$



In [8]:
def waveage(H, U, grav=9.80665):
    '''
    The pseudo wave age can be expressed in terms of significant wave height and surface wind speed.
    '''

    grav2 = grav**2
    hs2 = np.square(H) 
    u4 = np.power(U,4)
    tmp = np.divide(hs2*grav2,u4)
    eps = 3.25*np.power(tmp,0.31)
    
    return eps


def waveperiod(H, U, grav=9.80665):
    '''
    Wave age from GA-2, Govindan et al.
    '''

    eps = waveage(H, U, grav=9.80665)
    period = (((eps-(5.78))/(eps+(U/(H*((U/H)+H)))))+(H+(5.70)))
    
    return period

In [9]:
T = waveperiod(wh,ws)
print T
print len(T),len(wh)

[5.1940376  5.56986478 5.31512902 ... 5.57214165 4.4818197  5.22972275]
2998 2998


In [10]:
days2wh = wh[days] #wave height for each satellite pass
days2T = T[days]

print len(days2wh)
print len(days2T)

2639
2639


In [13]:
days2wh

array([1.36339998, 1.351125  , 1.34530771, ..., 2.64020023, 1.34984618,
       0.8967273 ])

In [14]:
days9wh = np.mean(wh[days])